In [1]:
import random
import csv
import os
import pickle
import sys
os.chdir('Resources/')

In [2]:
def power(b, p, m):
    b = b % m
    if p == 0:
        return 1
    j = power(b, p // 2, m)
    j = (j * j) % m
    if p % 2 == 1:
        j = (j * b) % m
    return j

In [3]:
def load_key(filename):
    with open(filename, "rb") as f:
        return pickle.load(f)

In [4]:
su_rsa_1_public_key = load_key("17_SU_RSA_1_Public_Key.pkl")
su_rsa_1_public_key

(4740866271843932821, 4936349149105973713)

In [5]:
input_filename = '18_(SYS)_SU_User_Data.csv'  # Your input file

In [ ]:
def encrypt(message, public_key):
    e, n = public_key
    encrypted_message = [pow(ord(char), e, n) for char in message]
    return encrypted_message

def encrypt_csv_file(input_filename, public_key):
    with open(input_filename, 'r') as f:
        reader = csv.reader(f)
        data = "\n".join([",".join(row) for row in reader])  # Join all rows into a single string
    
    encrypted_data = encrypt(data, public_key)
    
    return encrypted_data

encrypted_data = encrypt_csv_file(input_filename, su_rsa_1_public_key)
print(encrypted_data, '\n', len(encrypted_data))

def decrypt(private_key, encrypted_message):
    d, n = private_key
    decrypted_message = ''.join([chr(pow(char, d, n)) for char in encrypted_message])
    return decrypted_message

sys_rsa_1_private_key = load_key("16_SYS_RSA_1_Private_Key.pkl")

decrypted_data = decrypt(sys_rsa_1_private_key, encrypted_data)
print(decrypted_data)

[434633643918176515, 3372882327922177629, 1106448204126423065, 534664260735500318, 1106448204126423065, 3034407771811500895, 1106448204126423065, 534664260735500318, 4484170622493776446, 1985238862797609425, 1106448204126423065, 3034407771811500895, 434633643918176515, 1985238862797609425, 1106448204126423065, 534664260735500318, 1106448204126423065, 3034407771811500895, 1106448204126423065, 534664260735500318, 434633643918176515, 1985238862797609425, 1106448204126423065, 1985238862797609425, 1106448204126423065, 3034407771811500895, 1106448204126423065, 534664260735500318] 
 28
56,1,2,140,250,1,2,150,0,2,1


In [7]:
def encrypt_data_with_aes(data, key):
    encrypted_data = "".join(chr(ord(c) ^ key) for c in data)
    return encrypted_data

In [8]:
def encrypt_aes_key_with_rsa(aes_key, public_key):
    e, n = public_key
    return power(aes_key, e, n)

In [9]:
def encrypt_csv_file(input_filename, public_key):
    aes_key = random.randint(1, 255)  # Simple key for XOR-based encryption

    with open(input_filename, 'r', encoding='utf-8') as f:
        reader = csv.reader(f)
        data = "\n".join([",".join(row) for row in reader])

    encrypted_data = encrypt_data_with_aes(data, aes_key)
    encrypted_aes_key = encrypt_aes_key_with_rsa(aes_key, public_key)

    with open("19_(SYS)_SU_Transmitted_User_Data.csv", 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow([encrypted_aes_key])
        writer.writerow([encrypted_data])

    return encrypted_aes_key, encrypted_data

In [10]:
encrypted_aes_key, encrypted_data = encrypt_csv_file(input_filename, su_rsa_1_public_key)
print("File encrypted successfully. Data saved in '18_(SYS)_SU_User_Data.csv'.")

print(encrypted_aes_key, encrypted_data, len(encrypted_data))

File encrypted successfully. Data saved in '18_(SYS)_SU_User_Data.csv'.
4181640919787612689 yz`}`~`}x|`~y|`}`~`}y|`|`~`} 28


In [11]:
su_rsa_2_private_key = load_key("16_SU_RSA_2_Private_Key.pkl")
su_rsa_2_private_key

(455635778824898537, 4088522690914825687)

In [12]:
su_rsa_2_public_key = load_key("16_SU_RSA_2_Public_Key.pkl")
su_rsa_2_public_key

(2122126697722114073, 4088522690914825687)

In [13]:
def split_message(message, max_size):
    encoded = message.encode()
    chunks = [encoded[i:i + max_size] for i in range(0, len(encoded), max_size)]
    return [int.from_bytes(chunk, 'big') for chunk in chunks]

In [14]:
def sign_message(message, private_key, n):
    d, n = private_key
    max_bytes = (n.bit_length() // 8) - 1  # Ensure it fits within `n`
    message_chunks = split_message(message, max_bytes)
    
    signatures = [power(chunk, d, n) for chunk in message_chunks]  # Sign each chunk
    return signatures

In [15]:
signatures = sign_message(str(encrypted_aes_key) + '\n' + encrypted_data, su_rsa_2_private_key, su_rsa_2_public_key[1])
signatures

[3362278497443556638,
 2921472321343000950,
 2021250671343620426,
 1520114869881349655,
 3313927496720597972,
 4077870409221888809,
 3109189109144729207,
 3460634547077689931]

In [16]:
with open("19_(SYS)_SU_Transmitted_User_Data.csv", 'a', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow([signatures])